In [36]:
import sys
sys.path.append('../')

from model import *
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pickle as pkl
import matplotlib.pyplot as plt
import sklearn
import numpy as np

### Data

In [62]:
with open('../../data/ChildWelfare/X_preprocess.pkl', 'rb') as handle:
    X,screener_ids,refer_ids,Y_obs,D,Y_serv,Y_sub,colnames = pkl.load(handle)

In [38]:
X.shape

(46544, 216)

In [40]:
#add intercept column if needed
#X = np.append(X,np.ones((X.shape[0],1)),axis=1)
#X.shape

In [41]:
#drop instances if expert assessed less than a certain number of cases

drop_experts = []
for num in screener_ids:

    if screener_ids.count(num) < 10:

        drop_experts.append(num)

In [42]:
drop_idx = []
for index, elem in enumerate(screener_ids):
    if elem in drop_experts:
        drop_idx.append(index)

In [43]:
X = np.delete(X,drop_idx,axis=0)
Y_serv = np.delete(Y_serv,drop_idx,axis=0)
Y_sub = np.delete(Y_sub,drop_idx,axis=0)
Y_obs = np.delete(Y_obs,drop_idx,axis=0)
D = np.delete(D,drop_idx,axis=0)
refer_ids = np.delete(refer_ids,drop_idx,axis=0)
screener_ids = np.delete(screener_ids,drop_idx,axis=0)

In [45]:
D = D.reshape((D.shape[0],))
Y_obs = Y_obs.reshape((Y_obs.shape[0],))

In [46]:
target = pd.DataFrame({'D': D, 'Y1': Y_obs, 'Y2': Y_serv, 'Y3': Y_sub})

Split data in a 80% train, 20% test

In [47]:
cov_train, cov_test, tar_train, tar_test, nur_train, nur_test = train_test_split(pd.DataFrame(X), target, pd.Series(screener_ids), test_size = 0.2, random_state = 42)

### Modelling

In [48]:
# Model's characteristics
params = {'layers': []} # If = [] equivalent to a simple logistic regression

# Amalgation parameters
rho = 0.05 # Control which point to consider from a confience point of view
pi_1 = 4.0 # Control criterion on centre mass metric
pi_2 = 0.8 # Control criterion on opposing metric
tau = 1.0  # Balance between observed and expert labels

##### 1. Train on decision

This model models the nurse decision based on covariates

In [49]:
for l1_penalty in [ 1.0, 10.0, 20.0, 30.0, 50.0]:
    try:
        model = BinaryMLP(**params)
        model = model.fit(cov_train, tar_train['D'], nur_train, l1_penalty = l1_penalty)
        break
    except Exception as e:
        print(e, l1_penalty)
        pass









  0%|          | 0/1000 [00:00<?, ?it/s]







Loss: 0.627:   0%|          | 0/1000 [00:00<?, ?it/s]







Loss: 0.627:   0%|          | 1/1000 [00:00<05:32,  3.00it/s]







Loss: 0.609:   0%|          | 1/1000 [00:00<05:32,  3.00it/s]







Loss: 0.609:   0%|          | 2/1000 [00:00<05:35,  2.98it/s]







Loss: 0.595:   0%|          | 2/1000 [00:01<05:35,  2.98it/s]







Loss: 0.595:   0%|          | 3/1000 [00:01<05:32,  2.99it/s]







Loss: 0.589:   0%|          | 3/1000 [00:01<05:32,  2.99it/s]







Loss: 0.589:   0%|          | 4/1000 [00:01<05:35,  2.97it/s]







Loss: 0.583:   0%|          | 4/1000 [00:01<05:35,  2.97it/s]







Loss: 0.583:   0%|          | 5/1000 [00:01<06:06,  2.72it/s]







Loss: 0.578:   0%|          | 5/1000 [00:02<06:06,  2.72it/s]







Loss: 0.578:   1%|          | 6/1000 [00:02<05:58,  2.77it/s]







Loss: 0.571:   1%|          | 6/1000 [00:02<05:58,  2.77it/s]







Loss: 0.571:   1%|          | 7/1000 [00:02<05:53,  2.8

Loss: 0.549:   4%|▍         | 43/1000 [00:15<05:40,  2.81it/s]







Loss: 0.549:   4%|▍         | 44/1000 [00:15<05:33,  2.86it/s]







Loss: 0.548:   4%|▍         | 44/1000 [00:16<05:33,  2.86it/s]







Loss: 0.548:   4%|▍         | 45/1000 [00:16<05:38,  2.82it/s]







Loss: 0.549:   4%|▍         | 45/1000 [00:16<05:38,  2.82it/s]







Loss: 0.549:   5%|▍         | 46/1000 [00:16<05:32,  2.87it/s]







Loss: 0.549:   5%|▍         | 46/1000 [00:16<05:32,  2.87it/s]







Loss: 0.549:   5%|▍         | 47/1000 [00:16<05:25,  2.92it/s]







Loss: 0.549:   5%|▍         | 47/1000 [00:17<05:25,  2.92it/s]







Loss: 0.549:   5%|▍         | 48/1000 [00:17<05:17,  2.99it/s]







Loss: 0.550:   5%|▍         | 48/1000 [00:17<05:17,  2.99it/s]







Loss: 0.550:   5%|▍         | 49/1000 [00:17<05:18,  2.98it/s]







Loss: 0.550:   5%|▍         | 49/1000 [00:17<05:18,  2.98it/s]







Loss: 0.550:   5%|▌         | 50/1000 [00:17<05:16,  3.00it/s]







Loss: 0.548:   5%|▌ 

Loss: 0.549:   9%|▊         | 87/1000 [00:30<05:49,  2.62it/s]







Loss: 0.549:   9%|▊         | 87/1000 [00:31<05:49,  2.62it/s]







Loss: 0.549:   9%|▉         | 88/1000 [00:31<05:52,  2.59it/s]







Loss: 0.549:   9%|▉         | 88/1000 [00:31<05:52,  2.59it/s]







Loss: 0.549:   9%|▉         | 89/1000 [00:31<05:51,  2.59it/s]







Loss: 0.548:   9%|▉         | 89/1000 [00:31<05:51,  2.59it/s]







Loss: 0.548:   9%|▉         | 90/1000 [00:31<05:57,  2.55it/s]







Loss: 0.550:   9%|▉         | 90/1000 [00:32<05:57,  2.55it/s]







Loss: 0.550:   9%|▉         | 91/1000 [00:32<05:56,  2.55it/s]







Loss: 0.549:   9%|▉         | 91/1000 [00:32<05:56,  2.55it/s]







Loss: 0.549:   9%|▉         | 92/1000 [00:32<05:50,  2.59it/s]







Loss: 0.548:   9%|▉         | 92/1000 [00:32<05:50,  2.59it/s]







Loss: 0.548:   9%|▉         | 93/1000 [00:32<05:45,  2.62it/s]







Loss: 0.548:   9%|▉         | 93/1000 [00:33<05:45,  2.62it/s]







Loss: 0.548:   9%|▉ 

Loss: 0.550:  13%|█▎        | 130/1000 [00:47<05:20,  2.71it/s]







Loss: 0.548:  13%|█▎        | 130/1000 [00:47<05:20,  2.71it/s]







Loss: 0.548:  13%|█▎        | 131/1000 [00:47<05:16,  2.75it/s]







Loss: 0.549:  13%|█▎        | 131/1000 [00:47<05:16,  2.75it/s]







Loss: 0.549:  13%|█▎        | 132/1000 [00:47<05:06,  2.83it/s]







Loss: 0.549:  13%|█▎        | 132/1000 [00:48<05:06,  2.83it/s]







Loss: 0.549:  13%|█▎        | 133/1000 [00:48<04:59,  2.90it/s]







Loss: 0.549:  13%|█▎        | 133/1000 [00:48<04:59,  2.90it/s]







Loss: 0.549:  13%|█▎        | 134/1000 [00:48<04:54,  2.94it/s]







Loss: 0.547:  13%|█▎        | 134/1000 [00:48<04:54,  2.94it/s]







Loss: 0.547:  14%|█▎        | 135/1000 [00:48<04:53,  2.94it/s]







Loss: 0.549:  14%|█▎        | 135/1000 [00:49<04:53,  2.94it/s]







Loss: 0.549:  14%|█▎        | 136/1000 [00:49<04:50,  2.97it/s]







Loss: 0.548:  14%|█▎        | 136/1000 [00:49<04:50,  2.97it/s]







Loss: 

Loss: 0.549:  17%|█▋        | 173/1000 [01:02<05:01,  2.74it/s]







Loss: 0.548:  17%|█▋        | 173/1000 [01:03<05:01,  2.74it/s]







Loss: 0.548:  17%|█▋        | 174/1000 [01:03<04:54,  2.81it/s]







Loss: 0.549:  17%|█▋        | 174/1000 [01:03<04:54,  2.81it/s]







Loss: 0.549:  18%|█▊        | 175/1000 [01:03<04:49,  2.85it/s]







Loss: 0.550:  18%|█▊        | 175/1000 [01:03<04:49,  2.85it/s]







Loss: 0.550:  18%|█▊        | 176/1000 [01:03<04:43,  2.91it/s]







Loss: 0.549:  18%|█▊        | 176/1000 [01:04<04:43,  2.91it/s]







Loss: 0.549:  18%|█▊        | 177/1000 [01:04<04:38,  2.95it/s]







Loss: 0.549:  18%|█▊        | 177/1000 [01:04<04:38,  2.95it/s]







Loss: 0.549:  18%|█▊        | 178/1000 [01:04<04:35,  2.98it/s]







Loss: 0.550:  18%|█▊        | 178/1000 [01:04<04:35,  2.98it/s]







Loss: 0.550:  18%|█▊        | 179/1000 [01:04<04:37,  2.96it/s]







Loss: 0.550:  18%|█▊        | 179/1000 [01:05<04:37,  2.96it/s]







Loss: 

...

Loss: 0.687:   4%|▍         | 40/1000 [00:13<05:08,  3.11it/s]









Loss: 0.687:   4%|▍         | 41/1000 [00:13<05:04,  3.15it/s]









Loss: 0.686:   4%|▍         | 41/1000 [00:13<05:04,  3.15it/s]









Loss: 0.686:   4%|▍         | 42/1000 [00:13<05:09,  3.09it/s]









Loss: 0.685:   4%|▍         | 42/1000 [00:14<05:09,  3.09it/s]









Loss: 0.685:   4%|▍         | 43/1000 [00:14<05:08,  3.10it/s]









Loss: 0.686:   4%|▍         | 43/1000 [00:14<05:08,  3.10it/s]









Loss: 0.686:   4%|▍         | 44/1000 [00:14<05:08,  3.10it/s]









Loss: 0.685:   4%|▍         | 44/1000 [00:14<05:08,  3.10it/s]









Loss: 0.685:   4%|▍         | 45/1000 [00:14<05:04,  3.14it/s]









Loss: 0.686:   4%|▍         | 45/1000 [00:15<05:04,  3.14it/s]









Loss: 0.686:   5%|▍         | 46/1000 [00:15<05:06,  3.11it/s]









Loss: 0.688:   5%|▍         | 46/1000 [00:15<05:06,  3.11it/s]









Loss: 0.688:   5%|▍         | 47/1000 [00:15<05:03,  3.14it/s]



Loss: 0.686:   8%|▊         | 80/1000 [00:26<04:46,  3.21it/s]









Loss: 0.686:   8%|▊         | 81/1000 [00:26<04:44,  3.23it/s]









Loss: 0.686:   8%|▊         | 81/1000 [00:26<04:44,  3.23it/s]









Loss: 0.686:   8%|▊         | 82/1000 [00:26<04:42,  3.25it/s]









Loss: 0.688:   8%|▊         | 82/1000 [00:26<04:42,  3.25it/s]









Loss: 0.688:   8%|▊         | 83/1000 [00:26<04:41,  3.26it/s]









Loss: 0.686:   8%|▊         | 83/1000 [00:27<04:41,  3.26it/s]









Loss: 0.686:   8%|▊         | 84/1000 [00:27<04:40,  3.27it/s]









Loss: 0.686:   8%|▊         | 84/1000 [00:27<04:40,  3.27it/s]









Loss: 0.686:   8%|▊         | 85/1000 [00:27<04:39,  3.27it/s]









Loss: 0.687:   8%|▊         | 85/1000 [00:27<04:39,  3.27it/s]









Loss: 0.687:   9%|▊         | 86/1000 [00:27<04:38,  3.28it/s]









Loss: 0.686:   9%|▊         | 86/1000 [00:28<04:38,  3.28it/s]









Loss: 0.686:   9%|▊         | 87/1000 [00:28<04:40,  3.25it/s]



Loss: 0.687:  12%|█▏        | 120/1000 [00:38<04:46,  3.07it/s]









Loss: 0.687:  12%|█▏        | 121/1000 [00:38<04:44,  3.09it/s]









Loss: 0.686:  12%|█▏        | 121/1000 [00:39<04:44,  3.09it/s]









Loss: 0.686:  12%|█▏        | 122/1000 [00:39<04:41,  3.12it/s]









Loss: 0.686:  12%|█▏        | 122/1000 [00:39<04:41,  3.12it/s]









Loss: 0.686:  12%|█▏        | 123/1000 [00:39<04:43,  3.10it/s]









Loss: 0.686:  12%|█▏        | 123/1000 [00:39<04:43,  3.10it/s]









Loss: 0.686:  12%|█▏        | 124/1000 [00:39<04:47,  3.05it/s]









Loss: 0.687:  12%|█▏        | 124/1000 [00:40<04:47,  3.05it/s]









Loss: 0.687:  12%|█▎        | 125/1000 [00:40<04:43,  3.09it/s]









Loss: 0.686:  12%|█▎        | 125/1000 [00:40<04:43,  3.09it/s]









Loss: 0.686:  13%|█▎        | 126/1000 [00:40<04:40,  3.11it/s]









Loss: 0.687:  13%|█▎        | 126/1000 [00:40<04:40,  3.11it/s]









Loss: 0.687:  13%|█▎        | 127/1000 [00:40<04:39

Loss: 0.686:  16%|█▌        | 160/1000 [00:51<04:37,  3.03it/s]









Loss: 0.685:  16%|█▌        | 160/1000 [00:51<04:37,  3.03it/s]









Loss: 0.685:  16%|█▌        | 161/1000 [00:51<04:37,  3.02it/s]









Loss: 0.685:  16%|█▌        | 161/1000 [00:52<04:37,  3.02it/s]









Loss: 0.685:  16%|█▌        | 162/1000 [00:52<04:33,  3.07it/s]









Loss: 0.689:  16%|█▌        | 162/1000 [00:52<04:33,  3.07it/s]









Loss: 0.689:  16%|█▋        | 163/1000 [00:52<04:30,  3.10it/s]









Loss: 0.686:  16%|█▋        | 163/1000 [00:52<04:30,  3.10it/s]









Loss: 0.686:  16%|█▋        | 164/1000 [00:52<04:28,  3.12it/s]









Loss: 0.686:  16%|█▋        | 164/1000 [00:53<04:28,  3.12it/s]









Loss: 0.686:  16%|█▋        | 165/1000 [00:53<04:33,  3.06it/s]









Loss: 0.686:  16%|█▋        | 165/1000 [00:53<04:33,  3.06it/s]









Loss: 0.686:  17%|█▋        | 166/1000 [00:53<04:30,  3.08it/s]









Loss: 0.685:  17%|█▋        | 166/1000 [00:53<04:30

Loss: 0.687:  20%|█▉        | 199/1000 [01:04<04:15,  3.14it/s]









Loss: 0.687:  20%|██        | 200/1000 [01:04<04:18,  3.09it/s]









Loss: 0.686:  20%|██        | 200/1000 [01:04<04:18,  3.09it/s]









Loss: 0.686:  20%|██        | 201/1000 [01:04<04:16,  3.11it/s]









Loss: 0.685:  20%|██        | 201/1000 [01:05<04:16,  3.11it/s]









Loss: 0.685:  20%|██        | 202/1000 [01:05<04:16,  3.11it/s]









Loss: 0.686:  20%|██        | 202/1000 [01:05<04:16,  3.11it/s]









Loss: 0.686:  20%|██        | 203/1000 [01:05<04:12,  3.15it/s]









Loss: 0.687:  20%|██        | 203/1000 [01:05<04:12,  3.15it/s]









Loss: 0.687:  20%|██        | 204/1000 [01:05<04:16,  3.11it/s]









Loss: 0.686:  20%|██        | 204/1000 [01:06<04:16,  3.11it/s]









Loss: 0.686:  20%|██        | 205/1000 [01:06<04:15,  3.11it/s]









Loss: 0.686:  20%|██        | 205/1000 [01:06<04:15,  3.11it/s]









Loss: 0.686:  21%|██        | 206/1000 [01:06<04:13

In [51]:
Y_pred_h_test = model.predict(cov_test)

In [59]:
print("Y1", roc_auc_score(tar_test['Y1'], Y_pred_h_test))

print("Y2", roc_auc_score(tar_test['Y2'], Y_pred_h_test))

print("Y3", roc_auc_score(tar_test['Y3'], Y_pred_h_test))

print("D", roc_auc_score(tar_test['D'], Y_pred_h_test))

Y1 0.6898219116873691
Y2 0.8268652950840352
Y3 0.9600674945742804
D 0.7733231205027429


##### 2. Agreement computation 

Measure of agreeability are estimated in a cross validation fashion on the train set.

In [57]:
# Fold evaluation of influences
folds, predictions, influence = influence_cv(BinaryMLP, cov_train, tar_train['D'], nur_train, params = params, fit_params = {"l1_penalty": 25})













  0%|          | 0/1000 [00:00<?, ?it/s]











Loss: 0.695:   0%|          | 0/1000 [00:00<?, ?it/s]











Loss: 0.695:   0%|          | 1/1000 [00:00<05:08,  3.24it/s]











Loss: 0.693:   0%|          | 1/1000 [00:00<05:08,  3.24it/s]











Loss: 0.693:   0%|          | 2/1000 [00:00<04:49,  3.45it/s]











Loss: 0.692:   0%|          | 2/1000 [00:00<04:49,  3.45it/s]











Loss: 0.692:   0%|          | 3/1000 [00:00<04:37,  3.60it/s]











Loss: 0.693:   0%|          | 3/1000 [00:01<04:37,  3.60it/s]











Loss: 0.693:   0%|          | 4/1000 [00:01<04:29,  3.70it/s]











Loss: 0.694:   0%|          | 4/1000 [00:01<04:29,  3.70it/s]











Loss: 0.694:   0%|          | 5/1000 [00:01<04:23,  3.78it/s]











Loss: 0.692:   0%|          | 5/1000 [00:01<04:23,  3.78it/s]











Loss: 0.692:   1%|          | 6/1000 [00:01<04:16,  3.88it/s]











Loss: 0.693:   1%|          | 6/1000 [00:01<04:16,  3.88it/s]








Loss: 0.692:   4%|▎         | 37/1000 [00:08<03:39,  4.38it/s]











Loss: 0.692:   4%|▎         | 37/1000 [00:09<03:39,  4.38it/s]











Loss: 0.692:   4%|▍         | 38/1000 [00:09<03:41,  4.35it/s]











Loss: 0.693:   4%|▍         | 38/1000 [00:09<03:41,  4.35it/s]











Loss: 0.693:   4%|▍         | 39/1000 [00:09<03:39,  4.38it/s]











Loss: 0.692:   4%|▍         | 39/1000 [00:09<03:39,  4.38it/s]











Loss: 0.692:   4%|▍         | 40/1000 [00:09<03:42,  4.32it/s]











Loss: 0.692:   4%|▍         | 40/1000 [00:09<03:42,  4.32it/s]











Loss: 0.692:   4%|▍         | 41/1000 [00:09<03:41,  4.32it/s]











Loss: 0.693:   4%|▍         | 41/1000 [00:10<03:41,  4.32it/s]











Loss: 0.693:   4%|▍         | 42/1000 [00:10<03:42,  4.31it/s]











Loss: 0.691:   4%|▍         | 42/1000 [00:10<03:42,  4.31it/s]











Loss: 0.691:   4%|▍         | 43/1000 [00:10<03:41,  4.31it/s]











Loss: 0.692:   4%|▍         | 43/1000 

Loss: 0.692:   7%|▋         | 74/1000 [00:17<03:22,  4.57it/s]











Loss: 0.693:   7%|▋         | 74/1000 [00:17<03:22,  4.57it/s]











Loss: 0.693:   8%|▊         | 75/1000 [00:17<03:22,  4.56it/s]











Loss: 0.692:   8%|▊         | 75/1000 [00:17<03:22,  4.56it/s]











Loss: 0.692:   8%|▊         | 76/1000 [00:17<03:21,  4.60it/s]











Loss: 0.693:   8%|▊         | 76/1000 [00:17<03:21,  4.60it/s]











Loss: 0.693:   8%|▊         | 77/1000 [00:17<03:21,  4.58it/s]











Loss: 0.692:   8%|▊         | 77/1000 [00:18<03:21,  4.58it/s]











Loss: 0.692:   8%|▊         | 78/1000 [00:18<03:20,  4.60it/s]











Loss: 0.694:   8%|▊         | 78/1000 [00:18<03:20,  4.60it/s]











Loss: 0.694:   8%|▊         | 79/1000 [00:18<03:20,  4.60it/s]











Loss: 0.692:   8%|▊         | 79/1000 [00:18<03:20,  4.60it/s]











Loss: 0.692:   8%|▊         | 80/1000 [00:18<03:19,  4.61it/s]











Loss: 0.692:   8%|▊         | 80/1000 

Loss: 0.692:  11%|█         | 111/1000 [00:25<03:19,  4.45it/s]











Loss: 0.692:  11%|█         | 111/1000 [00:25<03:19,  4.45it/s]











Loss: 0.692:  11%|█         | 112/1000 [00:25<03:17,  4.49it/s]











Loss: 0.692:  11%|█         | 112/1000 [00:26<03:17,  4.49it/s]











Loss: 0.692:  11%|█▏        | 113/1000 [00:26<03:15,  4.54it/s]











Loss: 0.692:  11%|█▏        | 113/1000 [00:26<03:15,  4.54it/s]











Loss: 0.692:  11%|█▏        | 114/1000 [00:26<03:20,  4.42it/s]











Loss: 0.693:  11%|█▏        | 114/1000 [00:26<03:20,  4.42it/s]











Loss: 0.693:  12%|█▏        | 115/1000 [00:26<03:16,  4.50it/s]











Loss: 0.693:  12%|█▏        | 115/1000 [00:26<03:16,  4.50it/s]











Loss: 0.693:  12%|█▏        | 116/1000 [00:26<03:14,  4.54it/s]











Loss: 0.692:  12%|█▏        | 116/1000 [00:26<03:14,  4.54it/s]











Loss: 0.692:  12%|█▏        | 117/1000 [00:26<03:16,  4.50it/s]











Loss: 0.692:  12%|█▏     

ValueError: Architecture leads to singular weights matrix for last layer: Use another architecture or increase l1_penalty.

In [ ]:
# Compute metrics agreeability
center_metric, opposing_metric = compute_agreeability(influence)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(center_metric, opposing_metric)

In [ ]:
# Apply criteria on amalgamation
high_conf = (predictions > (1 - rho)) | (predictions < rho)
high_agr = (center_metric > pi_1) & (opposing_metric > pi_2) & high_conf
high_agr_correct = ((predictions - tar_train['D']).abs() < rho) & high_agr

In [ ]:
plt.hist(predictions)

In [ ]:
print('high_conf:', sum(high_conf))
print('high_agr:', sum(high_agr))
print('high_agr_correct:', sum(high_agr_correct))

In [ ]:
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(tar_train['D'], predictions, n_bins=7)
plt.plot(prob_true,prob_pred, marker='o', linewidth=1, label='logreg')


In [ ]:
# Create amalgamated labels
tar_train['Ya'] = tar_train['Y1'].copy()
tar_train['Ya'][high_agr_correct] = (1 - tau) * tar_train['Y1'][high_agr_correct] \
                                    + tau * tar_train['D'][high_agr_correct]

In [ ]:
sum(tar_train['D']!=tar_train['Y1'])

In [ ]:
sum(tar_train['Ya']!=tar_train['Y1'])

In [ ]:
#index for selective labels
index_amalg = [i==1.0 for i in tar_train['D']] | high_agr_correct

##### 3. Updated model

In [28]:
#model with selective labels
model = BinaryMLP(**params)
model = model.fit(cov_train[index_amalg], tar_train[index_amalg]['Ya'], nur_train[index_amalg], l1_penalty = l1_penalty)

Loss: 0.616: 100%|██████████| 1/1 [00:00<00:00,  5.10it/s]


In [29]:
Y_pred_amalg_test = model.predict(cov_test)

In [30]:
# #model without selective labels
# model = BinaryMLP(**params)
# model = model.fit(cov_train, tar_train['Ya'], nur_train[index_amalg])

In [ ]:
print("Y1", roc_auc_score(tar_test['Y1'], Y_pred_amalg_test))

print("Y2", roc_auc_score(tar_test['Y2'], Y_pred_amalg_test))

print("Y3", roc_auc_score(tar_test['Y3'], Y_pred_amalg_test))

print("D", roc_auc_score(tar_test['D'], Y_pred_amalg_test))

##### 4. Train on observed data

In [35]:
model = BinaryMLP(**params)
model = model.fit(cov_train[tar_train['D']==1], tar_train['Y1'][tar_train['D']==1], nur_train, l1_penalty = l1_penalty)

Loss: 0.616: 100%|██████████| 1/1 [00:00<00:00,  5.30it/s]


In [36]:
Y_pred_obs_test = model.predict(cov_test)

In [60]:
print("Y1", roc_auc_score(tar_test['Y1'], Y_pred_obs_test))

print("Y2", roc_auc_score(tar_test['Y2'], Y_pred_obs_test))

print("Y3", roc_auc_score(tar_test['Y3'], Y_pred_obs_test))

print("D", roc_auc_score(tar_test['D'], Y_pred_obs_test))

NameError: name 'Y_pred_obs_test' is not defined

## Precision/Accuracy

In [41]:
def eval_pred_outcome(Y_outcome_test, Y_pred_test, Y_h_test):
    Y_pred_test_scrin = Y_pred_test[Y_h_test==1]
    Y_outcome_test_scrin = Y_outcome_test[Y_h_test==1]
    #print(sum(Y_outcome_test_scrin))
    idx_25qrtl_pred = np.argsort(Y_pred_test_scrin)[::-1][:int(np.floor(p*len(Y_pred_test_scrin)))]
    #baseline
    baseline = sum(Y_outcome_test_scrin)/len(Y_outcome_test_scrin)
    #print(idx_25qrtl_pred )
    #print(sum(Y_outcome_test_scrin[idx_25qrtl_pred]))
    #precision and recall in top p% highsest scored screen-in
    precision_p = sum(Y_outcome_test_scrin[idx_25qrtl_pred])/len(idx_25qrtl_pred)
    recall_p = sum(Y_outcome_test_scrin[idx_25qrtl_pred])/sum(Y_outcome_test)
    
    return baseline, precision_p, recall_p

In [43]:
p=0.25
print('Eval resp. Y_obs')
print('f_y', eval_pred_outcome(np.array(tar_test['Y1']).flatten(), Y_pred_obs_test.flatten(), np.array(tar_test['D']==1).flatten()))
print('f_A', eval_pred_outcome(np.array(tar_test['Y1']).flatten(), Y_pred_amalg_test.flatten(), np.array(tar_test['D']==1).flatten()))
print('f_h', eval_pred_outcome(np.array(tar_test['Y1']).flatten(), Y_pred_h_test.flatten(), np.array(tar_test['D']==1).flatten()))

Eval resp. Y_obs
f_y (0.11727896649172385, 0.16155088852988692, 0.2617801047120419)
f_A (0.11727896649172385, 0.16155088852988692, 0.2617801047120419)
f_h (0.11727896649172385, 0.1704361873990307, 0.2761780104712042)


In [44]:
print('Eval resp. Y_serv')
print('f_y', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_obs_test.flatten(), np.array(tar_test['D']==1).flatten()))
print('f_A', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_amalg_test.flatten(), np.array(tar_test['D']==1).flatten()))
print('f_h', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_h_test.flatten(), np.array(tar_test['D']==1).flatten()))

Eval resp. Y_serv
f_y (0.4152200242228502, 0.3554119547657512, 0.21390374331550802)
f_A (0.4152200242228502, 0.3554119547657512, 0.21390374331550802)
f_h (0.4152200242228502, 0.6195476575121163, 0.37287311618862423)


In [45]:
print('Eval resp. Y_sub')
print('f_y', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_obs_test.flatten(), np.array(tar_test['D']==1).flatten()))
print('f_A', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_amalg_test.flatten(), np.array(tar_test['D']==1).flatten()))
print('f_h', eval_pred_outcome(np.array(tar_test['Y2']).flatten(), Y_pred_h_test.flatten(), np.array(tar_test['D']==1).flatten()))

Eval resp. Y_sub
f_y (0.4152200242228502, 0.3554119547657512, 0.21390374331550802)
f_A (0.4152200242228502, 0.3554119547657512, 0.21390374331550802)
f_h (0.4152200242228502, 0.6195476575121163, 0.37287311618862423)


In [ ]:
def eval_pred(Y_pred_test, Y_h, Y_ooh, Y_serv, Y_sub, refer_ids_test, refer_ids, p):
    #identify test sets
    dic_k_Y_pred = {}
    Y_h_test = np.array(Y_h[[(r in refer_ids_test) for r in refer_ids]])
    Y_h_test = np.transpose(Y_h_test)[0]
    
    #Y_pred for subset of cases that were actually screened-in
    Y_pred_test_scrin = Y_pred_test[Y_h_test==1]
    #top p% highest scored screened-in cases 
    idx_25qrtl_pred = np.argsort(Y_pred_test_scrin)[::-1][:int(np.floor(p*len(Y_pred_test_scrin)))]
    print('done with pre proc')
    base_ooh, precision_ooh, recall_ooh = eval_pred_outcome(idx_25qrtl_pred, Y_ooh, Y_h_test, refer_ids_test, refer_ids)
    print('done with ooh')
    base_serv, precision_serv, recall_serv = eval_pred_outcome(idx_25qrtl_pred, Y_serv, Y_h_test, refer_ids_test, refer_ids)
    base_sub, precision_sub, recall_sub = eval_pred_outcome(idx_25qrtl_pred, Y_sub, Y_h_test, refer_ids_test, refer_ids)
    
    dic_k_Y_pred['ooh'] = {'base': float(base_ooh), 'precision': float(precision_ooh), 'recall': float(recall_ooh)}
    print('done with dic ooh')
    dic_k_Y_pred['sub'] = {'base': float(base_sub), 'precision': float(precision_sub), 'recall': float(recall_sub)}
    dic_k_Y_pred['serv'] = {'base': float(base_serv), 'precision': float(precision_serv), 'recall': float(recall_serv)}
    
    return dic_k_Y_pred
    

In [ ]:
def eval_k_iter(M_results, Y_h, Y_ooh, Y_serv, Y_sub, p):
    K = len(M_results)
    dic_eval = {}
    
    for k in np.arange(K):
        dic_k = {}
        Y_pred_hybrid_test, Y_pred_obs_test, Y_pred_amalg_test, Y_pred_h_test, refer_ids_train, refer_ids_test = M_results[k]
        dic_k['Y_pred_obs_test'] = eval_pred(Y_pred_obs_test, Y_h, Y_ooh, Y_serv, Y_sub, refer_ids_test, refer_ids, p)
        print('done with Y_pred_obs_test')
        dic_k['Y_pred_h_test'] = eval_pred(Y_pred_h_test, Y_h, Y_ooh, Y_serv, Y_sub, refer_ids_test, refer_ids, p)
        print('done with Y_pred_h_test')
        dic_k['Y_pred_hybrid_test'] = eval_pred(Y_pred_hybrid_test, Y_h, Y_ooh, Y_serv, Y_sub, refer_ids_test, refer_ids, p)
        print('done with Y_pred_hyb_test')
        dic_k['Y_pred_amalg_test'] = eval_pred(Y_pred_amalg_test, Y_h, Y_ooh, Y_serv, Y_sub, refer_ids_test, refer_ids, p)
    
    
        dic_eval[k] = dic_k
    
    return dic_eval

In [ ]:
def plot_perf_bar(dic_eval, models = ['Y_pred_obs_test', 'Y_pred_h_test',
                                     'Y_pred_hybrid_test', 'Y_pred_amalg_test'], 
                  plot_labels = [ '$f_y$', '$f_h$', '$f_\mathcal{hyb}$',  '$f_\mathcal{A}$' ] ,
                  labels = ['ooh', 'sub', 'serv'], y_max=1.05, legend_appear=True,
                  metric ='precision', fig_title='fig_noname'):
    
    K = len(dic_eval.keys())
    
    n_labels = len(labels)
    
    fig, ax = plt.subplots(figsize=(10,5))
    index = np.arange(n_labels)
    bar_width = 0.15
    opacity = 0.8
    colors = ['green', 'darkred', 'yellow', 'blue']
    hatches = ['//', '-', '\\', '.']
    
    
    m_idx =0
    for m in models:
        plot_mean = [0]*n_labels
        plot_std = [0]*n_labels
        
        l_idx = 0
        for l in labels:
            metric_list = [0]*K
            if metric == 'precision':
                for k in range(K):
                    metric_list[k] = dic_eval[k][m][l]['precision']
            
            if metric == 'recall':
                for k in range(K):
                    metric_list[k] = dic_eval[k][m][l]['recall']
            plot_mean[l_idx] = np.mean(metric_list)  
            plot_std[l_idx] = np.std(metric_list)  
            l_idx += 1
        rects = plt.bar(index+m_idx*bar_width, plot_mean, yerr= plot_std,
            alpha=opacity, color=colors[m_idx], capsize=7,  width=.1, hatch = hatches[m_idx],
                            label=plot_labels[m_idx])        
            
            
        m_idx+=1                
    
    
    for n in index[:-1]:
        if metric=='precision':
            t= 0.8
        else:
            t=0.7
        plt.axvline(x=n+t, color='grey', linestyle='--')
        
    if metric=='precision':
        plot_mean = [0]*n_labels
        plot_std = [0]*n_labels
        l_idx = 0
        for l in labels:
            metric_list = [0]*K
            for k in range(K):
                metric_list[k] = dic_eval[k][m][l]['base']
            plot_mean[l_idx] = np.mean(metric_list)  
            plot_std[l_idx] = np.std(metric_list) 
            l_idx +=1
        rects = plt.bar(index+len(models)*bar_width, plot_mean, yerr= plot_std,
            alpha=opacity, color='grey',capsize=7,  width=.1,label='Overall prev.')     

    plt.ylabel(metric,fontsize=25)
    #plt.ylim((0,1.05))
    #plt.title('Performance of outcomes by model',fontsize=18)
    plt.xticks(index + 1.5*bar_width, ('OOH', 'Substantiated', 'Services'),fontsize=18)
    plt.yticks(fontsize=16)
    if legend_appear==True:
        plt.legend(loc = 'upper left', fontsize=18)
    plt.ylim(0.0,y_max)
    plt.tight_layout()

#     plt.savefig(fig_title, dpi=300,
#             orientation='portrait' )
    plt.show()